# pandas_ml

In [1]:
import seaborn as sns

In [2]:
iris =sns.load_dataset("iris")
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


- 説明/目的変数を別オブジェクトとして指定しなきゃいけないので煩雑
- sklearnは素晴らしいけどpandasが普及したのでnumpy basedの出力がストレスフル。
- sklearn使ってるとサブモジュールのimportが超多いのでめんどい。

そこで**pandas_ml**

ぶっちゃけると作者の[blog](http://sinhrks.hatenablog.com/entry/2015/03/19/234031)や[silde](https://speakerdeck.com/sinhrks/pandas-plus-scikit-learn-equals-pandas-ml-at-pydata-dot-tokyo-meetup-number-6-lightning-talk)を見た方がいいです。

In [3]:
import pandas_ml as pdml

In [4]:
df = pdml.ModelFrame(iris,target="species")
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


ModelFrameってクラスに突っ込みます。見た目は変わらないです。

In [5]:
print("iris: ",type(iris))
print("df: ",type(df))

iris:  <class 'pandas.core.frame.DataFrame'>
df:  <class 'pandas_ml.core.frame.ModelFrame'>


In [6]:
print (set(dir(df)) - set(dir(iris)))

{'_tree', 'predict_log_proba', '_decomposition', 'decision', '_wrap_probability', 'lm', 'multiclass', 'target', 'fit', '_cluster', 'data', 'target_name', 'pp', 'log_proba', '_semi_supervised', '_neighbors', 'neural_network', '_cross_validation', '_wrap_predicted', '_isotonic', 'feature_extraction', 'linear_model', '_mixture', 'ensemble', '_call', '_qda', 'crv', 'tree', '_DATA_NAME', '_concat_target', '_gaussian_process', 'mixture', 'has_target', 'decomposition', '_learning_curve', 'fit_predict', 'predicted', 'inverse_transform', 'random_projection', 'predict_proba', 'svm', 'preprocessing', 'cluster', '_naive_bayes', '_manifold', 'metrics', 'learning_curve', '_maybe_convert_data', 'predict', '_get_method_mapper', '_internal_caches', 'manifold', '_svm', '_grid_search', '_kernel_approximation', 'has_multi_targets', 'cls', '_covariance', 'proba', '_ensemble', 'transform', '_dummy', '_preprocessing', 'naive_bayes', 'grid_search', 'cross_validation', '_linear_model', '_pipeline', 'xgboost', 

色々メソッドが増えてます。  
基本的にsklearnのサブモジュールのものなので適当に探してください。  
Jupyterなら補完が効くので楽です。  

## ポイント1
ModelFrameにtargetを指定してあげると、説明変数と目的変数をメタデータとして持たせられる。

In [9]:
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [7]:
df.data.head()

,sepal_length,sepal_width,petal_length,petal_width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [21]:
df.target.head()

0    setosa
1    setosa
2    setosa
3    setosa
4    setosa
Name: species, dtype: object

## ポイント2
サブモジュールを呼ばなくてよい。

頻出する機能はpandas_mlの内部でサブパッケージを呼んでくれるので、自分でimport書かなくてよい。

In [10]:
df.data=df.data.preprocessing.normalize()
df.head()

,species,sepal_length,sepal_width,petal_length,petal_width
0,setosa,0.803773,0.551609,0.220644,0.031521
1,setosa,0.828133,0.507020,0.236609,0.033801
2,setosa,0.805333,0.548312,0.222752,0.034269
3,setosa,0.800030,0.539151,0.260879,0.034784
4,setosa,0.790965,0.569495,0.221470,0.031639


モデルですら大体よべる。

In [11]:
clf=df.svm.SVC()
clf

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

ModelFrameにfitやpredict,transformなども実装されているので、df側からモデルを指定して各種実行可能。

In [12]:
df[:-10].fit(clf)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [13]:
df[-10:].predict(clf)

140    versicolor
141    versicolor
142    versicolor
143    versicolor
144    versicolor
145    versicolor
146    versicolor
147    versicolor
148    versicolor
149    versicolor
dtype: object

全部が実装されているわけではないので、utils.shuffleなどは使えないが、pandasの機能でなんとかなる事が多い。

In [14]:
df.sample(frac=1).reset_index(drop=True).head()

,species,sepal_length,sepal_width,petal_length,petal_width
0,setosa,0.817689,0.517314,0.250313,0.033375
1,virginica,0.724153,0.325344,0.566728,0.220394
2,versicolor,0.764350,0.355818,0.513959,0.158141
3,versicolor,0.740886,0.331740,0.552900,0.187986
4,setosa,0.784175,0.566349,0.246870,0.058087


fitで渡すインスタンスは別にpandas_mlから呼び出すものに限らず、sklearn対応ならなんでもよい。  
サードパーティすらOK。

In [15]:
from tensorflow.contrib import learn as skflow

In [16]:
DNN=skflow.TensorFlowDNNClassifier(n_classes=3,hidden_units=[100,100,100],steps=1000)

In [17]:
#DNNはtargetを数値でしか受け付けないので変換しておく
df2=df.copy()
df2.target=df2.target.map({"setosa":0,"versicolor":1,"virginica":2})

In [18]:
df2[:-10].fit(DNN)

Step #100, epoch #20, avg. train loss: 0.52047
Step #200, epoch #40, avg. train loss: 0.25982
Step #300, epoch #60, avg. train loss: 0.17636
Step #400, epoch #80, avg. train loss: 0.12365
Step #500, epoch #100, avg. train loss: 0.11646
Step #600, epoch #120, avg. train loss: 0.09757
Step #700, epoch #140, avg. train loss: 0.10262
Step #800, epoch #160, avg. train loss: 0.09943
Step #900, epoch #180, avg. train loss: 0.10087
Step #1000, epoch #200, avg. train loss: 0.09907


TensorFlowDNNClassifier(batch_size=32, class_weight=None, clip_gradients=5.0,
            config=None, continue_training=False, dropout=None,
            hidden_units=[100, 100, 100], learning_rate=0.1, n_classes=3,
            optimizer='Adagrad', steps=1000, verbose=1)

In [19]:
df2[-10:].predict(DNN)

140    2
141    2
142    2
143    2
144    2
145    2
146    2
147    2
148    2
149    2
dtype: int64